## The required format for the GLAD program is:
```
First line:
    <numGivenLabels> <numLabelers> <numTasks> <Prior p(Z=1)>
  Following <numGivenLabels> lines:
    <taskId> <labelerId> <label:0|1>
```
0: Not a Disease (Includes "Phenotype")
<br>
1: Disease (Includes "Both")

I think task refers to the word in the dataset, and then labeler ID and the label. So if we have 20000 words in M2C and 5 label, the output of this program should have 100000 lines.

In [1]:
dataset_folder = 'dataset/Mark2Cure_citizen_phenotype/'

In [2]:
citizens = ['1', '2', '3', '4', '5']

In [3]:
filename = 'M2C{}-Disease_Phen_Preliminary_Run_1st_pass.txt'

In [4]:
text = ''

In [70]:
## Make vocab
vocab = {}
reverse_vocab = {}
sentences, labels = [], []
count_vocab = dict()
path = dataset_folder + filename.format('1')
i = 0
total_lines = 0
with open(path) as f:
    lines = f.read().split('\n')
    sent_temp, labels_temp = [], []
    for line in range(len(lines)):
        try:
            word = lines[line].split()[0].lower()
            if word == '.':
                labels.append(labels_temp)
                sent_temp, labels_temp = [], []
            elif word not in '.,:-+=_\\|/<>()!@#$%^&*':
                labels_temp.append(lines[line].split()[-1])
                c = count_vocab.get(word, 0)
                count_vocab[word] = c + 1
                if word not in vocab:
                    vocab[word] = i
                    reverse_vocab[i] = word
                    i += 1
        except:
            continue

In [136]:
len(count_vocab)

2234

In [145]:
# make bigram vocab:
## Make vocab
bigram_vocab = {}
reverse_bigram_vocab = {}
path = dataset_folder + filename.format('1')
i = 0


def make_bigrams(path):
    all_lines = ['<pad>']
    all_labels = ['O']
    bigram_lines = []
    bigram_labels = []
    i = 0
    wordidx = 0
    with open(path) as f:
        lines = f.read().split('\n')
        for line in lines:
            if line == '':
                continue
            label = line.split()[-1]
            line = line.split()[0].lower()
            if line not in ',:-+=_\\|/<>()!@#$%^&*':
                all_lines.append(line+'_{}'.format(wordidx))
                wordidx += 1
                all_labels.append(label)
                if line == '.':
                    all_lines.append('<pad>')
                    all_labels.append('O')

    flag = False #because we need <pad> before beginning the sentence
    for idx in range(len(all_lines) - 1):
        if all_lines[idx].split('_')[0] == '.':
            continue
        elif all_lines[idx+1].split('_')[0] == '.':
            continue

        bigram = (all_lines[idx], all_lines[idx+1])
        bigram_label = all_labels[idx+1]

        if bigram not in bigram_vocab:
            bigram_vocab[bigram] = i
            reverse_bigram_vocab[i] = bigram
            i += 1

        bigram_lines.append(bigram)
        bigram_labels.append(bigram_label)
    return bigram_lines, bigram_labels

In [146]:
# make 5 arrays of text
anno_bigrams, anno_labels = [], []
for citizen in citizens:
    path = dataset_folder + filename.format(citizen)
    bigrams, labels = make_bigrams(path)
    anno_bigrams.append(bigrams)
    anno_labels.append(labels)

In [147]:
text = '{} {} {} {} {} {}\n'.format(len(bigram_vocab)*len(anno_bigrams), len(anno_bigrams), len(bigram_vocab), 2, 0.5, 0.5)

In [148]:
for bigram in range(len(anno_bigrams[0])):
    for annotator in range(len(anno_bigrams)):
        label = '1'
        if anno_labels[annotator][bigram] == 'O':
            label = '0'
        text = text + '{} {} {}\n'.format(bigram_vocab[anno_bigrams[annotator][bigram]], annotator, label)

In [149]:
with open('data_phenotype_bigrams.txt', 'w') as f:
    f.write(text)

## The following code is to convert the alpha, beta and label csv to readable formats AFTER we run GLAD

### Make sure you have the proper files in the proper folder before running these hacky snippets

In [150]:
results_folder = 'Results for bigram/'
files = ['beta.csv', 'label.csv']

In [154]:
dict_betas = {}
text += '{}\t{}\n'.format('Word', 'Difficulty')
with open(results_folder+'beta.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line:
            arr = line.split(',')
            num = arr[0]
            if num == '# id':
                continue
            word = reverse_bigram_vocab[int(num)-1][1].split('_')[0]
            beta_temp = dict_betas.get(word, 0.0)
            dict_betas[word] = beta_temp + float(arr[1])
for word, _ in dict_betas.items():
    text += '{}\t{}\n'.format(word, dict_betas[word]/count_vocab[word])
with open(results_folder+'beta_bigram.csv','w') as g:
    g.write(text)

In [155]:
text = '{}\t{}\t{}\n'.format('Word', 'P(None)', 'P(Phenotype)')
p_0 = {}
p_1 = {}
with open(results_folder+'label.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line:
            arr = line.split(',')
            num = arr[0]
            if num == '# id':
                continue
            word = reverse_bigram_vocab[int(num)-1][1].split('_')[0]
            
            p_0_value = p_0.get(word, 0.0)
            p_0[word] = p_0_value + float(arr[1])
            
            p_1_value = p_1.get(word, 0.0)
            p_1[word] = p_1_value + float(arr[2])
            
for word, _ in p_0.items():
    text += '{}\t{}\t{}\n'.format(word, p_0[word]/count_vocab[word], p_1[word]/count_vocab[word])
    
with open(results_folder+'label_bigram.csv','w') as g:
    g.write(text)